Considerations of team size in a conversation with ChatGPT: https://chatgpt.com/c/66db5e28-0fbc-8005-8867-0a3691f6f5f7

This conversation was from sometime in early 2024 IIRC.

The goal was to consider the probability of fielding a Mythic team of 20 on any given night given roster sizes from 25-28. A quick consideration of this is often wrong, as the math is intuitive.

Towards the end of the conversation we get into considering attendance rates for individuals instead of using a distribution. The compute time needed to do this for a team of 28 was too high for ChatGPT's servers at the time, so I had to run it locally. This is the code.

I recently needed to re-run this so I figured I'd toss it into this repo.

In [2]:
import numpy as np
import pandas as pd
from itertools import combinations

# Function to generate attendance rates with a wider standard deviation
def generate_attendance_rates(n, mean=0.9, std=0.15, low=0.6, high=1.0):
    attendance_rates = np.random.normal(mean, std, n)
    return np.clip(attendance_rates, low, high)

# Function to calculate the exact probability of exactly k attendees
def exact_k_attendees(attendance_rates, k):
    n = len(attendance_rates)
    total_prob = 0

    # Generate all combinations of k attendees
    for attendees in combinations(range(n), k):
        prob_attend = 1
        prob_not_attend = 1
        for i in range(n):
            if i in attendees:
                prob_attend *= attendance_rates[i]
            else:
                prob_not_attend *= (1 - attendance_rates[i])
        total_prob += prob_attend * prob_not_attend

    return total_prob

# Generate sample data for a team of 28 people
n_people = 28
team_attendance_rates = generate_attendance_rates(n_people)

# Create a DataFrame to visualize the attendance rates
team_attendance_df = pd.DataFrame({
    'Person': [f'Person {i+1}' for i in range(n_people)],
    'Attendance Rate': team_attendance_rates
})

# Function to calculate cumulative probabilities of at least k attendees
def cumulative_probabilities(team_attendance_rates, min_attendees, max_attendees):
    cumulative_probs = []
    for k in range(min_attendees, max_attendees + 1):
        cumulative_prob = sum(exact_k_attendees(team_attendance_rates, i) for i in range(k, max_attendees + 1))
        cumulative_probs.append(cumulative_prob)
    return cumulative_probs

# Calculate cumulative probabilities for at least 20 to 28 attendees
cumulative_probabilities_wide = cumulative_probabilities(team_attendance_rates, 20, n_people)

# Display the attendance rates and the cumulative probabilities
print("Team Attendance Rates:\n", team_attendance_df)
print("\nCumulative Probabilities for at least 20 to 28 attendees:\n", cumulative_probabilities_wide)


Team Attendance Rates:
        Person  Attendance Rate
0    Person 1         0.885756
1    Person 2         0.970914
2    Person 3         1.000000
3    Person 4         1.000000
4    Person 5         1.000000
5    Person 6         0.722025
6    Person 7         0.916659
7    Person 8         1.000000
8    Person 9         0.727240
9   Person 10         0.864397
10  Person 11         0.827404
11  Person 12         0.908021
12  Person 13         0.659109
13  Person 14         0.880363
14  Person 15         0.864167
15  Person 16         0.748744
16  Person 17         0.600000
17  Person 18         1.000000
18  Person 19         1.000000
19  Person 20         1.000000
20  Person 21         0.884948
21  Person 22         1.000000
22  Person 23         0.823824
23  Person 24         1.000000
24  Person 25         0.902833
25  Person 26         0.895804
26  Person 27         0.688861
27  Person 28         0.900179

Cumulative Probabilities for at least 20 to 28 attendees:
 [np.float64(0.998